# HW 4 - Neural POS Tagger

In this exercise, you are going to build a set of deep learning models on part-of-speech (POS) tagging using Tensorflow and Keras. Tensorflow is a deep learning framwork developed by Google, and Keras is a frontend library built on top of Tensorflow (or Theano, CNTK) to provide an easier way to use standard layers and networks.

To complete this exercise, you will need to build deep learning models for POS tagging in Thai using NECTEC's ORCHID corpus. You will build one model for each of the following type:

- Neural POS Tagging with Word Embedding using Fixed / non-Fixed Pretrained weights
- Neural POS Tagging with Viterbi / Marginal CRF

Pretrained word embeddding are already given for you to use (albeit, a very bad one). Optionally, you can use your best pretrained word embeddding from previous exercise.

We also provide the code for data cleaning, preprocessing and some starter code for keras in this notebook but feel free to modify those parts to suit your needs. You can also complete this exercise using only Tensorflow (without using Keras). Feel free to use additional libraries (e.g. scikit-learn) as long as you have a model for each type mentioned above.

### Don't forget to shut down your instance on Gcloud when you are not using it ###

## 1. Setup and Preprocessing

We use POS data from [ORCHID corpus](https://www.nectec.or.th/corpus/index.php?league=pm), which is a POS corpus for Thai language.
A method used to read the corpus into a list of sentences with (word, POS) pairs have been implemented already. The example usage has shown below.
We also create a word vector for unknown word by random.

In [1]:
from data.orchid_corpus import get_sentences
import numpy as np
import numpy.random
import keras.preprocessing
np.random.seed(42)

Using TensorFlow backend.


In [69]:
unk_emb =np.random.randn(32)
train_data = get_sentences('train')
test_data = get_sentences('test')
print(train_data[0] , len(train_data[0]))
print(train_data[1], len(train_data[1]))
print(train_data[-1], len(train_data[-1]))
print(len(train_data))

max_len = max([len(line) for line in train_data])
print(max_len)

[('การ', 'FIXN'), ('ประชุม', 'VACT'), ('ทาง', 'NCMN'), ('วิชาการ', 'NCMN'), ('<space>', 'PUNC'), ('ครั้ง', 'CFQC'), ('ที่ 1', 'DONM')] 7
[('โครงการวิจัยและพัฒนา', 'NCMN'), ('อิเล็กทรอนิกส์', 'NCMN'), ('และ', 'JCRG'), ('คอมพิวเตอร์', 'NCMN')] 4
[('อาจ', 'XVMM'), ('มี', 'VSTA'), ('โรคมะเร็ง', 'NCMN'), ('ได้', 'XVAE'), ('<space>', 'PUNC'), ('มาก', 'ADVN'), ('กว่า', 'JCMP'), ('<space>', 'PUNC'), ('1', 'DCNM'), ('<space>', 'PUNC'), ('ชนิด', 'CLTV'), ('<space>', 'PUNC'), ('ดัง', 'RPRE'), ('ตาราง', 'NCMN'), ('ที่ 2', 'DONM')] 15
18500
102


Next, we load pretrained weight embedding using pickle. The pretrained weight is a dictionary which map a word to its embedding.

In [29]:
import pickle
fp = open('basic_ff_embedding.pt', 'rb')
embeddings = pickle.load(fp)
fp.close()

The given code below generates an indexed dataset(each word is represented by a number) for training and testing data. The index 0 is reserved for padding to help with variable length sequence. (Additionally, You can read more about padding here [https://machinelearningmastery.com/data-preparation-variable-length-input-sequences-sequence-prediction/])

## 2. Prepare Data

In [30]:
word_to_idx ={}
idx_to_word ={}
label_to_idx = {}
for sentence in train_data:
    for word,pos in sentence:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)+1
            idx_to_word[word_to_idx[word]] = word
        if pos not in label_to_idx:
            label_to_idx[pos] = len(label_to_idx)+1
word_to_idx['UNK'] = len(word_to_idx)

n_classes = len(label_to_idx.keys())+1

This section is tweaked a little from the demo, word2features will return word index instead of features, and sent2labels will return a sequence of word indices in the sentence.

In [31]:
def word2features(sent, i, emb):
    word = sent[i][0]
    if word in word_to_idx :
        return word_to_idx[word]
    else :
        return word_to_idx['UNK']

def sent2features(sent, emb_dict):
    return np.asarray([word2features(sent, i, emb_dict) for i in range(len(sent))])

def sent2labels(sent):
    return numpy.asarray([label_to_idx[label] for (word, label) in sent],dtype='int32')

def sent2tokens(sent):
    return [word for (word, label) in sent]

In [61]:
sent2features(train_data[100], embeddings)

array([ 29, 327,   5, 328])

In [65]:
print(len(word_to_idx), len(label_to_idx), idx_to_word[29],idx_to_word[327],
     idx_to_word[5],idx_to_word[328])

15019 47 รัฐมนตรีว่าการ กระทรวงวิทยาศาสตร์เทคโนโลยีและการพลังงาน <space> ประธานกรรมการ


Next we create train and test dataset, then we use keras to post-pad the sequence to max sequence with 0. Our labels are changed to a one-hot vector.

In [33]:
%%time
x_train = np.asarray([sent2features(sent, embeddings) for sent in train_data])
y_train = [sent2labels(sent) for sent in train_data]
y_train_temp = y_train
x_test = [sent2features(sent, embeddings) for sent in test_data]
y_test = [sent2labels(sent) for sent in test_data]

CPU times: user 399 ms, sys: 0 ns, total: 399 ms
Wall time: 395 ms


In [34]:
x_train=keras.preprocessing.sequence.pad_sequences(x_train, maxlen=None, dtype='int32', padding='post', truncating='pre', value=0.)
y_train=keras.preprocessing.sequence.pad_sequences(y_train, maxlen=None, dtype='int32', padding='post', truncating='pre', value=0.)
x_test=keras.preprocessing.sequence.pad_sequences(x_test, maxlen=102, dtype='int32', padding='post', truncating='pre', value=0.)
y_temp =[]
for i in range(len(y_train)):
    y_temp.append(np.eye(n_classes)[y_train[i]][np.newaxis,:])
y_train = np.asarray(y_temp).reshape(-1,102,n_classes)
del(y_temp)

In [9]:
print(x_train[100],x_train.shape)
print(y_train[100][3],y_train.shape)

[ 29 327   5 328   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0] (18500, 102)
[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] (18500, 102, 48)


## 3. Evaluate

Our output from keras is a distribution of problabilities on all possible label. outputToLabel will return an indices of maximum problability from output sequence.

evaluation_report is the same as in the demo

In [10]:
def outputToLabel(yt,seq_len):
    out = []
    for i in range(0,len(yt)):
        if(i==seq_len):
            break
        out.append(np.argmax(yt[i]))
    return out

In [11]:
import pandas as pd
from IPython.display import display

def evaluation_report(y_true, y_pred):
    # retrieve all tags in y_true
    tag_set = set()
    for sent in y_true:
        for tag in sent:
            tag_set.add(tag)
    for sent in y_pred:
        for tag in sent:
            tag_set.add(tag)
    tag_list = sorted(list(tag_set))
    
    # count correct points
    tag_info = dict()
    for tag in tag_list:
        tag_info[tag] = {'correct_tagged': 0, 'y_true': 0, 'y_pred': 0}

    all_correct = 0
    all_count = sum([len(sent) for sent in y_true])
    for sent_true, sent_pred in zip(y_true, y_pred):
        for tag_true, tag_pred in zip(sent_true, sent_pred):
            if tag_true == tag_pred:
                tag_info[tag_true]['correct_tagged'] += 1
                all_correct += 1
            tag_info[tag_true]['y_true'] += 1
            tag_info[tag_pred]['y_pred'] += 1
    accuracy = (all_correct / all_count) * 100
            
    # summarize and make evaluation result
    eval_list = list()
    for tag in tag_list:
        eval_result = dict()
        eval_result['tag'] = tag
        eval_result['correct_count'] = tag_info[tag]['correct_tagged']
        precision = (tag_info[tag]['correct_tagged']/tag_info[tag]['y_pred'])*100 if tag_info[tag]['y_pred'] else '-'
        recall = (tag_info[tag]['correct_tagged']/tag_info[tag]['y_true'])*100 if (tag_info[tag]['y_true'] > 0) else 0
        eval_result['precision'] = precision
        eval_result['recall'] = recall
        eval_result['f_score'] = (2*precision*recall)/(precision+recall) if (type(precision) is float and recall > 0) else '-'
        
        eval_list.append(eval_result)

    eval_list.append({'tag': 'accuracy=%.2f' % accuracy, 'correct_count': '', 'precision': '', 'recall': '', 'f_score': ''})
    
    df = pd.DataFrame.from_dict(eval_list)
    df = df[['tag', 'precision', 'recall', 'f_score', 'correct_count']]
    display(df)

## 4. Train a model

In [14]:
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense,GRU,Reshape,TimeDistributed,Bidirectional,Dropout,Masking
from keras_contrib.layers import CRF
from keras.optimizers import Adam

The model is this section is separated to two groups

- Neural POS Tagger (4.1)
- Neural CRF POS Tagger (4.2)

## 4.1.1 Neural POS Tagger  (Example)

We create a simple Neural POS Tagger as an example for you. This model dosen't use any pretrained word embbeding so it need to use Embedding layer to train the word embedding from scratch.

In [12]:
model = Sequential()
model.add(Embedding(len(word_to_idx),32,input_length=102,mask_zero=True))
model.add(Bidirectional(GRU(32, return_sequences=True)))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(n_classes,activation='softmax')))
model.summary()
adam  = Adam(lr=0.001)
model.compile(optimizer=adam,  loss='categorical_crossentropy', metrics=['categorical_accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 102, 32)           480608    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 102, 64)           12480     
_________________________________________________________________
dropout_1 (Dropout)          (None, 102, 64)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 102, 48)           3120      
Total params: 496,208
Trainable params: 496,208
Non-trainable params: 0
_________________________________________________________________


In [13]:
%%time
model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=1)

Epoch 1/10
18500/18500 [==============================] - 131s - loss: 1.8997 - categorical_accuracy: 0.5464   
Epoch 2/10
18500/18500 [==============================] - 119s - loss: 0.4277 - categorical_accuracy: 0.9024   
Epoch 3/10
18500/18500 [==============================] - 119s - loss: 0.2553 - categorical_accuracy: 0.9351   
Epoch 4/10
18500/18500 [==============================] - 119s - loss: 0.1997 - categorical_accuracy: 0.9461   
Epoch 5/10
18500/18500 [==============================] - 119s - loss: 0.1721 - categorical_accuracy: 0.9521   
Epoch 6/10
18500/18500 [==============================] - 119s - loss: 0.1563 - categorical_accuracy: 0.9552   
Epoch 7/10
18500/18500 [==============================] - 120s - loss: 0.1445 - categorical_accuracy: 0.9579   
Epoch 8/10
18500/18500 [==============================] - 119s - loss: 0.1361 - categorical_accuracy: 0.9600   
Epoch 9/10
18500/18500 [==============================] - 120s - loss: 0.1289 - categorical_accuracy: 0.

In [14]:
%%time
model.save_weights('/data/exp_pos_no_crf.h5')
#model.load_weights('/data/exp_pos_no_crf.h5')
y_pred=model.predict(x_test)
ypred = [outputToLabel(y_pred[i],len(y_test[i])) for i in range(len(y_pred))]
evaluation_report(y_test, ypred)

,tag,precision,recall,f_score,correct_count
0,1,99.8092,99.3758,99.5921,3662
1,2,94.8278,94.4714,94.6493,7792
2,3,91.062,96.5184,93.7108,16301
3,4,99.9689,99.3654,99.6662,12840
4,5,91.6667,98.5075,94.964,66
5,6,99.7817,87.5479,93.2653,457
6,7,97.6374,97.4026,97.5199,2025
7,8,67.3716,53.7349,59.7855,223
8,9,56.3725,62.5,59.2784,230
9,10,62.6316,42.5507,50.6742,357


CPU times: user 45.4 s, sys: 7.66 s, total: 53 s
Wall time: 18.5 s


## 4.1.2 Neural POS Tagger - Fix Weight

### #TODO 1
We would like you create a neural postagger model with keras with the pretrained word embedding as an input. The word embedding should be fixed across training time. To finish this excercise you must train the model and show the evaluation report with this model as shown in the example.

(You may want to read about Keras's Masking layer)

Optionally, you can use your own pretrained word embedding from previous homework

In [70]:
pre_em = []
pre_em.append(np.zeros(32))
for i in range(1,len(idx_to_word)+1):
    if(idx_to_word[i] in embeddings.keys()):
        pre_em.append(embeddings[idx_to_word[i]])
    else:
        pre_em.append(np.zeros(32))

In [71]:
len(pre_em)

15019

In [19]:
# Write your code here
model = Sequential()
model.add(Embedding(len(word_to_idx),32,input_length=102,mask_zero=True, weights=[np.array(pre_em)], trainable=False))
model.add(Bidirectional(GRU(32, return_sequences=True)))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(n_classes,activation='softmax')))
model.summary()
adam  = Adam(lr=0.001)
model.compile(optimizer=adam,  loss='categorical_crossentropy', metrics=['categorical_accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 102, 32)           480608    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 102, 64)           12480     
_________________________________________________________________
dropout_1 (Dropout)          (None, 102, 64)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 102, 48)           3120      
Total params: 496,208
Trainable params: 15,600
Non-trainable params: 480,608
_________________________________________________________________


In [20]:
%%time
model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=1)

Epoch 1/10
18500/18500 [==============================] - 119s - loss: 2.4451 - categorical_accuracy: 0.3027   
Epoch 2/10
18500/18500 [==============================] - 119s - loss: 2.0917 - categorical_accuracy: 0.3625   
Epoch 3/10
18500/18500 [==============================] - 119s - loss: 1.9125 - categorical_accuracy: 0.4373   
Epoch 4/10
18500/18500 [==============================] - 120s - loss: 1.7347 - categorical_accuracy: 0.5000   
Epoch 5/10
18500/18500 [==============================] - 120s - loss: 1.6138 - categorical_accuracy: 0.5380   
Epoch 6/10
18500/18500 [==============================] - 119s - loss: 1.5232 - categorical_accuracy: 0.5659   
Epoch 7/10
18500/18500 [==============================] - 120s - loss: 1.4557 - categorical_accuracy: 0.5867   
Epoch 8/10
18500/18500 [==============================] - 119s - loss: 1.4041 - categorical_accuracy: 0.6005   
Epoch 9/10
18500/18500 [==============================] - 119s - loss: 1.3650 - categorical_accuracy: 0.

In [21]:
%%time
model.save_weights('/data/fixw_pos_no_crf.h5')
#model.load_weights('/data/exp_pos_no_crf.h5')
y_pred=model.predict(x_test)
ypred = [outputToLabel(y_pred[i],len(y_test[i])) for i in range(len(y_pred))]
evaluation_report(y_test, ypred)

,tag,precision,recall,f_score,correct_count
0,1,94.7097,99.5929,97.0899,3670
1,2,64.1585,65.9554,65.0445,5440
2,3,54.5816,64.2963,59.042,10859
3,4,62.9213,85.2422,72.4004,11015
4,5,-,0,-,0
5,6,22.2222,0.383142,0.753296,2
6,7,93.981,85.6181,89.6048,1780
7,8,26.3158,4.81928,8.14664,20
8,9,16.1765,2.98913,5.04587,11
9,10,-,0,-,0


CPU times: user 45.6 s, sys: 7.3 s, total: 52.9 s
Wall time: 18.2 s


## 4.1.3 Neural POS Tagger - Trainable pretrained weight

### #TODO 2
We would like you create a neural postagger model with keras with the pretrained word embedding as an input. However The word embedding is trainable (not fixed). To finish this excercise you must train the model and show the evaluation report with this model as shown in the example.

Please note that the given pretrained word embedding only have weights for the vocabuary in BEST corpus from previous homework.

Optionally, you can use your own pretrained word embedding from previous homework.

In [17]:
# Write your code here
# Write your code here
model = Sequential()
model.add(Embedding(len(word_to_idx),32,input_length=102,mask_zero=True, weights=[np.array(pre_em)], trainable=True))
model.add(Bidirectional(GRU(32, return_sequences=True)))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(n_classes,activation='softmax')))
model.summary()
adam  = Adam(lr=0.001)
model.compile(optimizer=adam,  loss='categorical_crossentropy', metrics=['categorical_accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 102, 32)           480608    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 102, 64)           12480     
_________________________________________________________________
dropout_1 (Dropout)          (None, 102, 64)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 102, 48)           3120      
Total params: 496,208
Trainable params: 496,208
Non-trainable params: 0
_________________________________________________________________


In [25]:
%%time
model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=1)

Epoch 1/10
18500/18500 [==============================] - 120s - loss: 1.9630 - categorical_accuracy: 0.5061   
Epoch 2/10
18500/18500 [==============================] - 120s - loss: 0.5473 - categorical_accuracy: 0.8760   
Epoch 3/10
18500/18500 [==============================] - 121s - loss: 0.3129 - categorical_accuracy: 0.9233   
Epoch 4/10
18500/18500 [==============================] - 120s - loss: 0.2347 - categorical_accuracy: 0.9399   
Epoch 5/10
18500/18500 [==============================] - 120s - loss: 0.1970 - categorical_accuracy: 0.9474   
Epoch 6/10
18500/18500 [==============================] - 120s - loss: 0.1745 - categorical_accuracy: 0.9523   
Epoch 7/10
18500/18500 [==============================] - 121s - loss: 0.1603 - categorical_accuracy: 0.9551   
Epoch 8/10
18500/18500 [==============================] - 121s - loss: 0.1493 - categorical_accuracy: 0.9576   
Epoch 9/10
18500/18500 [==============================] - 121s - loss: 0.1418 - categorical_accuracy: 0.

In [26]:
%%time
model.save_weights('/data/nfixw_pos_no_crf.h5')
#model.load_weights('/data/exp_pos_no_crf.h5')
y_pred=model.predict(x_test)
ypred = [outputToLabel(y_pred[i],len(y_test[i])) for i in range(len(y_pred))]
evaluation_report(y_test, ypred)

,tag,precision,recall,f_score,correct_count
0,1,99.8911,99.5929,99.7418,3670
1,2,94.9155,93.9258,94.418,7747
2,3,91.199,96.6961,93.8671,16331
3,4,99.9379,99.5898,99.7636,12869
4,5,84.4156,97.0149,90.2778,65
5,6,99.7817,87.5479,93.2653,457
6,7,97.824,97.3064,97.5645,2023
7,8,69.209,59.0361,63.7191,245
8,9,70.3264,64.4022,67.234,237
9,10,63.4752,42.6698,51.0335,358


CPU times: user 46 s, sys: 7.4 s, total: 53.4 s
Wall time: 18.4 s


### #TODO 3
Compare the result between all neural tagger models in 4.1.x and provide a convincing reason and example for the result of these models (which model perform best or worst, why?)

(If you use your own weight please state so in the answer)

<b>Write your answer here :</b> 

Trainable Embbeding layers gave us the highest probability.


## 4.2.1 CRF Viterbi

Your next two tasks are to incorporate Conditional random fields (CRF) to your model. <b>You do not need to use pretrained weight</b>.

Keras already implement a CRF neural model for you. However, you need to use the official extension repository for Keras library, call keras-contrib. You should read about keras-contrib crf layer before attempt this exercise section

### #TODO 4
Use Keras-contrib CRF layer in your model. You should set the layer parameter so it can give the best performance on testing using <b>viterbi algorithm</b>. Your model must use crf for loss function and metric. CRF is quite complex compare to previous example model, so you should train it with more epoch, so it can converge.

To finish this excercise you must train the model and show the evaluation report with this model as shown in the example.

Do not forget to save this model weight.

In [18]:
from keras_contrib.layers import CRF
from keras.callbacks import ReduceLROnPlateau
from keras import regularizers

In [112]:
# Write your code here
# import keras.backend as K
# K.clear_session()
model = Sequential()
model.add(Embedding(len(word_to_idx),32,input_length=102,mask_zero=True))
# model.add(Masking(mask_value=0))
model.add(Bidirectional(GRU(32, return_sequences=True)))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(n_classes, activation='tanh')))
crf = CRF(n_classes,
#           learn_mode='join',
#           test_mode='viterbi',
#           sparse_target=True,
#           use_boundary=True,
#           use_bias=True,
#           activation='linear',
#           kernel_initializer='glorot_uniform',
#           chain_initializer='orthogonal',
#           bias_initializer='zeros',
#           boundary_initializer='zeros',
#           kernel_regularizer=regularizers.l1_l2(0.),
#           chain_regularizer=regularizers.l1_l2(0.),
#           boundary_regularizer=regularizers.l1_l2(0.),
#           bias_regularizer=regularizers.l1_l2(0.),
#           kernel_constraint=None,
#           chain_constraint=None,
#           boundary_constraint=None,
#           bias_constraint=None,
#           input_dim=None,
#           unroll=False
         )
model.add(crf)
model.summary()
adam  = Adam(lr=0.001)
model.compile(optimizer=adam,loss=crf.loss_function, metrics=[crf.accuracy])
# model.compile(optimizer=adam,  loss='categorical_crossentropy', metrics=['categorical_accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 102, 32)           480608    
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 102, 64)           12480     
_________________________________________________________________
dropout_19 (Dropout)         (None, 102, 64)           0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, 102, 48)           3120      
_________________________________________________________________
crf_8 (CRF)                  (None, 102, 48)           4752      
Total params: 500,960
Trainable params: 500,960
Non-trainable params: 0
_________________________________________________________________


In [114]:
model.fit(x_train,y_train,batch_size=128,epochs=20,verbose=1,shuffle=True)

Epoch 1/20
18500/18500 [==============================] - 86s - loss: 46.5739 - acc: 0.4348    
Epoch 2/20
18500/18500 [==============================] - 85s - loss: 45.0770 - acc: 0.8384    
Epoch 3/20
18500/18500 [==============================] - 85s - loss: 44.7255 - acc: 0.9087    
Epoch 4/20
18500/18500 [==============================] - 84s - loss: 44.6219 - acc: 0.9287    
Epoch 5/20
18500/18500 [==============================] - 85s - loss: 44.5770 - acc: 0.9370    
Epoch 6/20
18500/18500 [==============================] - 84s - loss: 44.5527 - acc: 0.9413    
Epoch 7/20
18500/18500 [==============================] - 83s - loss: 44.5360 - acc: 0.9442    
Epoch 8/20
18500/18500 [==============================] - 82s - loss: 44.5236 - acc: 0.9468    
Epoch 9/20
18500/18500 [==============================] - 85s - loss: 44.5133 - acc: 0.9491    
Epoch 10/20
18500/18500 [==============================] - 83s - loss: 44.5053 - acc: 0.9516    
Epoch 11/20
18500/18500 [==============

In [ ]:
%%time
model.fit(x_train,y_train,batch_size=128,epochs=5,verbose=1,shuffle=True)

In [118]:
%%time
model.save_weights('/data/crf_viterbi.h5')
#model.load_weights('/data/exp_pos_no_crf.h5')
y_pred=model.predict(x_test)
ypred = [outputToLabel(y_pred[i],len(y_test[i])) for i in range(len(y_pred))]
evaluation_report(y_test, ypred)

,tag,precision,recall,f_score,correct_count
0,1,99.8911,99.5929,99.7418,3670
1,2,94.4417,93.3196,93.8773,7697
2,3,90.3482,96.1632,93.1651,16241
3,4,99.9767,99.4351,99.7051,12849
4,5,95.6522,98.5075,97.0588,66
5,6,100,88.5057,93.9024,462
6,7,97.1689,97.4026,97.2856,2025
7,8,68.1416,55.6627,61.2732,231
8,9,78.6713,61.1413,68.8073,225
9,10,62.2776,41.7163,49.9643,350


CPU times: user 1min 10s, sys: 11.4 s, total: 1min 21s
Wall time: 27.8 s


In [82]:
for v in y_train_temp:
    print(v)

[1 2 3 3 4 5 6]
[3 3 7 3]
[3 4 8]
[9 4 6]
[10]
[10]
[3 6 4 6 4 3 4 8]
[3 4 9 4 8]
[3]
[11  4  3 10]
[10 12 13  1  2  3 14  1  2  3 14  3  4 14  3  3 14  1 13  3 15 15]
[14  1  2 16 13  3 17  4 18 19 13  3 20  9 14  1 13  3  7 13  3  4 14  4  1
  2  3 21  2 14  1  2 14  3]
[10  4 18 12  2 13  3 22 14  1  2  3  3  7  3]
[16  3 17 18 13  3 21 22 15 14  3]
[16 13  3  4  7  3 21 22 14  1  2  3 20 20  9]
[14  3  4  8  4 10  4 14  3  3  3 12  2 10 23]
[16  2  3  3 14  3  4  7  3 14  3  7  3  7  3]
[16  1  2 16 13  1  2 14  3 17]
[14  3  2  3 14  1  2  7  2 14  1  2 14 14  3  7  3]
[ 7 12  2 23 14  3  3  7  3]
[ 3  4 12  2 23  4 24  4 25 23  4  3  1  2 23  2  3 21 13  3 15]
[16 13  3  3  2 23 15]
[13  3 13  3]
[ 7 12  2  3 13  3]
[ 2 14  1  2  3 26 22]
[ 2  1  2  3  4  7  2  3  4 16 13  3  7  3 14 10]
[14  3  1  2  3 14  5 17  4 27 13  1 13  7  2 26 22 21  3 12  2  4  3  4  3
  4  7  3 14  3 17]
[18  2  2 14  3 28 13 14  3  7  3 29  4 21 13]
[ 2  2  4  7  2 26 22]
[16  3 14 27 13  3  4  7  3 1

## 4.2.2 CRF Marginal

### #TODO 5

Use Keras-contrib CRF layer in your model. You should set the layer parameter so it can give the best performance on testing using <b>marginal problabilities</b>. You <b>must not train the model</b> from scratch but use the pretrained weight from previous CRF Viterbi model.

To finish this excercise you must train the model and show the evaluation report with this model as shown in the example.

In [ ]:
# Write your code here
model = Sequential()
model.add(Embedding(len(word_to_idx),32,input_length=102,mask_zero=True))
# model.add(Masking(mask_value=0.))
model.add(Bidirectional(GRU(32, return_sequences=True)))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(n_classes, activation='sigmoid')))
crf = CRF(n_classes,
          learn_mode='marginal',
          test_mode='marginal',
          sparse_target=False,
          use_boundary=True,
          use_bias=True,
          activation='softmax',
          kernel_initializer='glorot_uniform',
          chain_initializer='orthogonal',
          bias_initializer='zeros',
          boundary_initializer='zeros',
          kernel_regularizer=None,
          chain_regularizer=None,
          boundary_regularizer=None,
          bias_regularizer=None,
          kernel_constraint=None,
          chain_constraint=None,
          boundary_constraint=None,
          bias_constraint=None,
          input_dim=None,
          unroll=False)
model.add(crf)
model.summary()
adam  = Adam(lr=0.001)
model.compile(optimizer=adam,  loss=crf.loss_function, metrics=[crf.accuracy])

In [ ]:
%%time
model.fit(x_train,y_train,batch_size=512,epochs=20,verbose=1,shuffle=True,validation_split=0.2)

### #TODO 6

Please pick the best example that can show the different between CRF that use viterbi and CRF that use marginal problabilities. Compare the result and provide a convincing reason. (which model perform better, why?)

<b>Write your answer here :</b>